# Chapter 18: Math Methods in DataFrames

In [1]:
import pandas as pd
import numpy as np

In [2]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/siena2018-pres.csv'
df = pd.read_csv(url, index_col=0)

In [3]:
def tweak_siena_pres(df):
    def int64_to_uint8(df_):
        cols = df_.select_dtypes('int64')
        return (df_
                .astype({col:'uint8' for col in cols}))


    return (df
     .rename(columns={'Seq.':'Seq'})    # 1 removes period from column name Eq.
     .rename(columns={k:v.replace(' ', '_') for k,v in
        {'Bg': 'Background',
         'PL': 'Party leadership', 'CAb': 'Communication ability',
         'RC': 'Relations with Congress', 'CAp': 'Court appointments',
         'HE': 'Handling of economy', 'L': 'Luck',
         'AC': 'Ability to compromise', 'WR': 'Willing to take risks',
         'EAp': 'Executive appointments', 'OA': 'Overall ability',
         'Im': 'Imagination', 'DA': 'Domestic accomplishments',
         'Int': 'Integrity', 'EAb': 'Executive ability',
         'FPA': 'Foreign policy accomplishments',
         'LA': 'Leadership ability',
         'IQ': 'Intelligence', 'AM': 'Avoid crucial mistakes',
         'EV': "Experts' view", 'O': 'Overall'}.items()})
     .astype({'Party':'category'})  # 2 sets the type of Party column to category
     .pipe(int64_to_uint8)  # 3 converts all the int64 columns to unsigned 8-bit columns
     .assign(Average_rank=lambda df_:(df_.select_dtypes('uint8') # 4 creates am average_rank column
                 .sum(axis=1).rank(method='dense').astype('uint8')),
             Quartile=lambda df_:pd.qcut(df_.Average_rank, 4,
                 labels='1st 2nd 3rd 4th'.split())
            )
    )

In [4]:
pres = tweak_siena_pres(df)

In [5]:
pres.head()

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
1,1,George Washington,Independent,7,7,1,10,1,6,2,...,1,1,1,2,2,1,2,1,1,1st
2,2,John Adams,Federalist,3,13,4,4,24,14,31,...,4,13,15,19,13,16,10,14,13,2nd
3,3,Thomas Jefferson,Democratic-Republican,2,2,14,1,8,5,14,...,7,20,4,6,9,7,5,5,5,1st
4,4,James Madison,Democratic-Republican,4,6,7,3,16,15,6,...,6,14,7,11,19,11,8,7,7,1st
5,5,James Monroe,Democratic-Republican,9,14,11,18,6,16,7,...,11,9,9,10,5,6,9,8,8,1st


## 18.1 Index Alignment

In [6]:
scores = (pres
            .loc[:,'Background':'Average_rank']
)

In [7]:
scores.head()

,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,...,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank
1,7,7,1,10,1,6,2,2,1,11,...,1,1,1,1,2,2,1,2,1,1
2,3,13,4,4,24,14,31,21,21,13,...,17,4,13,15,19,13,16,10,14,13
3,2,2,14,1,8,5,14,6,6,4,...,5,7,20,4,6,9,7,5,5,5
4,4,6,7,3,16,15,6,13,17,10,...,10,6,14,7,11,19,11,8,7,7
5,9,14,11,18,6,16,7,10,12,15,...,8,11,9,9,10,5,6,9,8,8


In [10]:
tmp_s = scores.iloc[2:4, :6]
tmp_s

,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks
3,2,2,14,1,8,5
4,4,6,7,3,16,15


In [8]:
s1 = scores.iloc[:3, :4]
s1

,Background,Imagination,Integrity,Intelligence
1,7,7,1,10
2,3,13,4,4
3,2,2,14,1


In [ ]:
s2 = scores.iloc[1:6, :5]
s2

,Background,Imagination,Integrity,Intelligence,Luck
2,3,13,4,4,24
3,2,2,14,1,8
4,4,6,7,3,16
5,9,14,11,18,6
6,1,9,6,5,29


In [ ]:
# only the overlapping rows (2 and 3) and columns (background through intelligence) get added together
s1 + s2

,Background,Imagination,Integrity,Intelligence,Luck
1,NaN,NaN,NaN,NaN,NaN
2,6.0,26.0,8.0,8.0,NaN
3,4.0,4.0,28.0,2.0,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN


In [12]:
pd.concat ([ scores.iloc [1:6 , :5]]*2)

,Background,Imagination,Integrity,Intelligence,Luck
2,3,13,4,4,24
3,2,2,14,1,8
4,4,6,7,3,16
5,9,14,11,18,6
6,1,9,6,5,29
2,3,13,4,4,24
3,2,2,14,1,8
4,4,6,7,3,16
5,9,14,11,18,6
6,1,9,6,5,29


In [13]:
s1

,Background,Imagination,Integrity,Intelligence
1,7,7,1,10
2,3,13,4,4
3,2,2,14,1


In [23]:
s2=scores.iloc[:3 , 2:6]
s2

,Integrity,Intelligence,Luck,Willing_to_take_risks
1,1,10,1,6
2,4,4,24,14
3,14,1,8,5


In [24]:
s1

,Background,Imagination,Integrity,Intelligence
1,7,7,1,10
2,3,13,4,4
3,2,2,14,1


In [25]:
s1.mul(s2)

,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks
1,NaN,NaN,1,100,NaN,NaN
2,NaN,NaN,16,16,NaN,NaN
3,NaN,NaN,196,1,NaN,NaN


In [26]:
s1.mul(s2, axis=0)

,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks
1,NaN,NaN,1,100,NaN,NaN
2,NaN,NaN,16,16,NaN,NaN
3,NaN,NaN,196,1,NaN,NaN


In [27]:
s1.mul(s2, axis='columns')

,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks
1,NaN,NaN,1,100,NaN,NaN
2,NaN,NaN,16,16,NaN,NaN
3,NaN,NaN,196,1,NaN,NaN


In [28]:
s1.mul(s2, axis='rows')

,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks
1,NaN,NaN,1,100,NaN,NaN
2,NaN,NaN,16,16,NaN,NaN
3,NaN,NaN,196,1,NaN,NaN
